# Ignore all this

In [117]:
cd /home/kirk/centristDataScience/data/results

/home/kirk/centristDataScience/data/results


In [154]:
import pandas as pd
import json

colorado_state_results = pd.read_csv('icpsr_clean.csv')

mov_under_five = 0
indep_double_figures = 0
indep_quarter = 0
bottom_quarter_turnout = 0
under_20_turnout = 0

def bottom_quarter_by_year(df):
    quarters = dict()
    years = df.year.unique()
    for i in years:
        quarters[str(i)] = df[df['year'] == i].total_votes.quantile(.25)
    return quarters

def check_district_turnouts(df, quarters_dict):
    total = 0
    for i in df.year.unique():
        t = df[df['year'] == i].total_votes.iloc[0]
        if t < quarters_dict[str(i)]:
            total += 1
    return total

def primaryTurnouts(p_dict, num, house_type):
    if house_type == 'senate':
        twelve = filter(lambda x: x['turnout_average'] <= 20.0,
                        filter(lambda x: x['district'].split(' ')[-1] == str(num), p_dict['primary_12_senate']))
        #four = filter(lambda x: x['turnout_average'] <= 10.0,
        #              filter(lambda x: x['district'].split(' ')[-1] == str(num), p_dict['primary_14_senate']))
    else:
        twelve = filter(lambda x: x['turnout_average'] <= 20.0,
                        filter(lambda x: x['district'].split(' ')[-1] == str(num), p_dict['primary_12_house']))
        #four = filter(lambda x: x['turnout_average'] <= 10.0,
        #              filter(lambda x: x['district'].split(' ')[-1] == str(num), p_dict['primary_14_house']))
    return len(twelve) * under_20_turnout #+ len(four) * under_20_turnout
    
        
        
def score_district(num, df, quarter_dict, house_type, p_dict):
    races = df[df['district_number'] == num]
    indeps = races[races['simple_party'] == 'indep']
    mov_below_five = races[races['percent_MoV'] <= 5.0]
    indep_double = indeps[indeps['candidate_percent'] >= 10.0]
    indep_25 = indeps[indeps['candidate_percent'] >= 25.0]
    turnouts_bottom_25 = check_district_turnouts(races, quarter_dict)
    under_10s = primaryTurnouts(p_dict, num, house_type)
    total_points = (len(mov_below_five) * mov_under_five) + (len(indep_double) * indep_double_figures) +\
        (turnouts_bottom_25 * bottom_quarter_turnout) + (len(indep_25) * indep_quarter) + under_10s
    return total_points


def score_districts(df, house_type):
    p_dict = json.load(open('primary_data.json', 'r'))
    districts = df.district_number.unique()
    quarters = bottom_quarter_by_year(df)
    scores = map(lambda x: {'district': x, 'score': score_district(x, df, quarters, house_type, p_dict)}, districts)
    return pd.DataFrame(scores).sort('score', ascending=False)

senate_races = colorado_state_results[colorado_state_results['senate_or_house'] == 8]
house_races = colorado_state_results[colorado_state_results['senate_or_house'] == 9]

In [155]:
p.keys()

[u'primary_12_senate', u'primary_12_house']

## Play with the inputs below to change ranking factor importance
## Press the play button at the top of the screen when finished to calculate rankings
* ###We call this "running a cell"

### ProTip: you can also press Shift+Enter

## Variable name meanings:

1. mov_under_five: percent margin of victory less than 5 points
2. indep_double_figures: independent candidate received double digit percentage of total vote
3. indep_quarter: independent candidate received more than 25% of total votes
4. bottom_quarter_turnout: total votes cast in bottom 25% percentile for colorado
5. under_20_turnout: percent registered voter turnout in 2012, 2014 primaries less then 20

In [156]:
mov_under_five = 2
indep_double_figures = 4
indep_quarter = 2
bottom_quarter_turnout = 2
under_20_turnout = 2


house_scores = score_districts(house_races, 'house')
senate_scores = score_districts(senate_races, 'senate')

## To see house district rankings, run the cell below

In [157]:
print house_scores

    district  score
16        17     72
49        50     66
35        36     58
1          2     56
64        65     54
29        30     54
7          8     50
26        27     48
33        34     48
41        42     48
4          5     46
31        32     46
30        31     44
6          7     44
3          4     42
34        35     42
10        11     40
50        51     38
18        19     38
42        43     38
46        47     38
8          9     38
2          3     36
21        22     36
20        21     34
28        29     32
12        13     30
51        52     30
43        44     30
45        46     30
..       ...    ...
57        58     26
22        23     26
62        63     24
11        12     24
14        15     22
44        45     22
37        38     20
59        60     20
52        53     20
54        55     18
55        56     18
32        33     18
15        16     18
17        18     18
56        57     16
36        37     16
19        20     16
48        49     14


## To see senate rankings, run the next one

In [158]:
print senate_scores

    district  score
22        11     34
6         12     32
31        31     28
26        26     26
12        19     24
1          5     22
34        34     22
9         16     20
25        25     20
32        32     20
7         13     18
29        29     18
20         3     18
19         2     18
16        24     18
24        20     16
33        33     16
0          4     14
15        23     12
4          9     12
28        28     12
5         10     12
30        30     10
2          6     10
21         8     10
23        15     10
11        18     10
10        17     10
8         14      8
13        21      8
18         1      6
27        27      6
3          7      4
14        22      4
17        35      2
